<a href="https://colab.research.google.com/github/aruntakhur/SitareUniversity/blob/main/MinMax_AlphaBeta_heuristic_AI_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic-Tac-Toe with Minimax and Alpha-Beta Pruning

This implementation uses depth-limited Minimax with a heuristic "expected utility" evaluator.

## Game Setup

- **'X'** is the maximizing player
- **'O'** is the minimizing player  
- **board** is a list of 9 elements: 'X', 'O', or ' ' (space) for empty
- **get_best_move(board, player, max_depth)** returns the best move index (0..8) for 'player' computed by Minimax with depth cutoff and heuristic estimate

## Heuristic Function

The heuristic used for cutoff evaluation:

- **Terminal states**: +100 for X win, -100 for O win, 0 for draw
- **Non-terminal nodes at cutoff**: Evaluate each line (row/col/diag):
  - If line contains only X and empty spaces: add positive weights
    - 1 X → +1 point
    - 2 X → +10 points  
  - If line contains only O and empty spaces: subtract similarly
  
This yields an estimate of "expected utility" indicating how favorable the board position is.

In [1]:
from typing import List, Tuple, Optional

LINES = [
    (0,1,2), (3,4,5), (6,7,8),  # rows
    (0,3,6), (1,4,7), (2,5,8),  # cols
    (0,4,8), (2,4,6)            # diags
]

def is_terminal(board: List[str]) -> Tuple[bool, Optional[str]]:
    """Return (True, 'X'/'O'/'Draw') if terminal, else (False, None)."""
    for a,b,c in LINES:
        if board[a] != ' ' and board[a] == board[b] == board[c]:
            return True, board[a]
    if ' ' not in board:
        return True, 'Draw'
    return False, None

def heuristic_expected_utility(board: List[str]) -> int:
    """
    Heuristic estimate of utility for non-terminal states.
    Positive => good for X (maximizer). Negative => good for O (minimizer).
    """
    score = 0
    for a,b,c in LINES:
        line = [board[a], board[b], board[c]]
        if line.count('X') > 0 and line.count('O') > 0:
            # contested line -> no immediate potential
            continue
        # only X and empties
        if line.count('O') == 0:
            nx = line.count('X')
            if nx == 1:
                score += 1        # small potential
            elif nx == 2:
                score += 10       # strong potential
        # only O and empties
        if line.count('X') == 0:
            no = line.count('O')
            if no == 1:
                score -= 1
            elif no == 2:
                score -= 10
    return score

def utility_value(winner: str) -> int:
    """Terminal utility values."""
    if winner == 'X':
        return 100
    if winner == 'O':
        return -100
    return 0  # draw

def available_moves(board: List[str]) -> List[int]:
    return [i for i, v in enumerate(board) if v == ' ']

def minimax(board: List[str],
            depth: int,
            max_depth: int,
            maximizing: bool,
            alpha: int = -10**9,
            beta: int = 10**9) -> Tuple[int, Optional[int]]:
    """
    Depth-limited Minimax with alpha-beta pruning.
    Returns (best_score, best_move_index).
    If best_move_index is None => no moves (terminal).
    """
    term, winner = is_terminal(board)
    if term:
        return utility_value(winner), None

    if depth >= max_depth:
        # Cutoff: return heuristic estimate (expected utility)
        return heuristic_expected_utility(board), None

    moves = available_moves(board)
    best_move = None

    if maximizing:
        best_score = -10**9
        for m in moves:
            board[m] = 'X'
            score, _ = minimax(board, depth+1, max_depth, False, alpha, beta)
            board[m] = ' '
            if score > best_score:
                best_score = score
                best_move = m
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break  # beta cut-off
        return best_score, best_move
    else:
        best_score = 10**9
        for m in moves:
            board[m] = 'O'
            score, _ = minimax(board, depth+1, max_depth, True, alpha, beta)
            board[m] = ' '
            if score < best_score:
                best_score = score
                best_move = m
            beta = min(beta, best_score)
            if beta <= alpha:
                break  # alpha cut-off
        return best_score, best_move

def get_best_move(board: List[str], player: str, max_depth: int = 4) -> Optional[int]:
    """
    Public helper to obtain best move for 'player' ('X' or 'O') with depth cutoff.
    max_depth: maximum search depth (0 means only heuristic evaluation).
    """
    maximizing = (player == 'X')
    score, move = minimax(board, depth=0, max_depth=max_depth, maximizing=maximizing)
    # Note: when cutoff occurs at root (max_depth==0), move can be None -> choose best heuristically
    if move is None:
        # fallback: choose move that maximizes heuristic locally
        best_move = None
        best_score = -10**9 if maximizing else 10**9
        for m in available_moves(board):
            board[m] = player
            s = heuristic_expected_utility(board)
            board[m] = ' '
            if maximizing and s > best_score:
                best_score, best_move = s, m
            if not maximizing and s < best_score:
                best_score, best_move = s, m
        return best_move
    return move

# ---------- Utilities for demonstration ----------
def pretty_print(board: List[str]):
    for r in range(3):
        print(' | '.join(board[3*r:3*r+3]))
        if r < 2:
            print('---------')

if __name__ == "__main__":
    # Example: partially-filled board
    # Indexes:
    # 0 1 2
    # 3 4 5
    # 6 7 8
    board = ['X', 'O', 'X',
             ' ', 'O', ' ',
             ' ', ' ', ' ']

    print("Current board:")
    pretty_print(board)
    print()

    # Get best move for X with depth limit 4
    best_move = get_best_move(board, 'X', max_depth=4)
    print(f"Best move for X (depth limit 4): {best_move}")
    if best_move is not None:
        board[best_move] = 'X'
        print("Board after recommended move:")
        pretty_print(board)


Current board:
X | O | X
---------
  | O |  
---------
  |   |  

Best move for X (depth limit 4): 7
Board after recommended move:
X | O | X
---------
  | O |  
---------
  | X |  


## Interactive Tic-Tac-Toe Game

Now let's create an interactive version where you can play against the AI!

In [2]:
def print_board_interactive(board: List[str]):
    """Display the board with position numbers for empty spaces."""
    print("\nCurrent Board:")
    print("-------------")
    for r in range(3):
        row = []
        for c in range(3):
            pos = 3*r + c
            if board[pos] == ' ':
                row.append(str(pos))  # Show position number for empty spaces
            else:
                row.append(board[pos])
        print(f" {row[0]} | {row[1]} | {row[2]} ")
        if r < 2:
            print("-----------")
    print()

def print_positions():
    """Show the position numbering system."""
    print("Position numbers:")
    print(" 0 | 1 | 2 ")
    print("-----------")
    print(" 3 | 4 | 5 ")
    print("-----------")
    print(" 6 | 7 | 8 ")
    print()

def get_human_move(board: List[str]) -> int:
    """Get valid move from human player."""
    while True:
        try:
            move = int(input("Enter your move (0-8): "))
            if move < 0 or move > 8:
                print("Please enter a number between 0 and 8.")
                continue
            if board[move] != ' ':
                print("That position is already taken! Choose another.")
                continue
            return move
        except ValueError:
            print("Please enter a valid number.")

In [3]:
def play_interactive_game():
    """Main game loop for human vs AI with proper terminal checking."""
    print("🎮 Welcome to Tic-Tac-Toe!")
    print("You are 'O' and the AI is 'X'")
    print_positions()
    
    # Choose who goes first
    while True:
        first = input("Who goes first? (h)uman or (a)i: ").lower()
        if first in ['h', 'human']:
            human_first = True
            break
        elif first in ['a', 'ai']:
            human_first = False
            break
        else:
            print("Please enter 'h' for human or 'a' for AI.")
    
    # Choose AI difficulty
    while True:
        try:
            difficulty = int(input("AI difficulty (1=Easy, 2=Medium, 3=Hard, 4=Expert): "))
            if 1 <= difficulty <= 4:
                max_depth = difficulty + 1  # depth 2-5
                break
            else:
                print("Please enter 1, 2, 3, or 4.")
        except ValueError:
            print("Please enter a valid number.")
    
    board = [' '] * 9
    current_player = 'O' if human_first else 'X'
    
    print(f"\n🚀 Game starting! {'You' if human_first else 'AI'} go first.")
    
    while True:
        print_board_interactive(board)
        
        # Check if game is over before making a move
        terminal, winner = is_terminal(board)
        if terminal:
            if winner == 'Draw':
                print("🤝 It's a draw!")
            elif winner == 'O':
                print("🎉 Congratulations! You won!")
            else:
                print("🤖 AI wins! Better luck next time.")
            break
        
        # Make move
        if current_player == 'O':  # Human turn
            print("Your turn (O):")
            move = get_human_move(board)
            board[move] = 'O'
            print(f"You played position {move}")
        else:  # AI turn
            print("AI is thinking...")
            move = get_best_move(board, 'X', max_depth)
            if move is not None:
                board[move] = 'X'
                print(f"AI played position {move}")
            else:
                print("AI couldn't find a move!")
                break
        
        # Check if game is over immediately after the move
        terminal, winner = is_terminal(board)
        if terminal:
            print_board_interactive(board)  # Show final board state
            if winner == 'Draw':
                print("🤝 It's a draw!")
            elif winner == 'O':
                print("🎉 Congratulations! You won!")
            else:
                print("🤖 AI wins! Better luck next time.")
            break
        
        # Switch players
        current_player = 'X' if current_player == 'O' else 'O'
    
    # Ask to play again
    while True:
        again = input("\nPlay again? (y/n): ").lower()
        if again in ['y', 'yes']:
            play_interactive_game()
            break
        elif again in ['n', 'no']:
            print("Thanks for playing! 👋")
            break
        else:
            print("Please enter 'y' for yes or 'n' for no.")

In [4]:
# Run this cell to start playing!
play_interactive_game()

🎮 Welcome to Tic-Tac-Toe!
You are 'O' and the AI is 'X'
Position numbers:
 0 | 1 | 2 
-----------
 3 | 4 | 5 
-----------
 6 | 7 | 8 

Please enter 'h' for human or 'a' for AI.

🚀 Game starting! You go first.

Current Board:
-------------
 0 | 1 | 2 
-----------
 3 | 4 | 5 
-----------
 6 | 7 | 8 

Your turn (O):
You played position 2

Current Board:
-------------
 0 | 1 | O 
-----------
 3 | 4 | 5 
-----------
 6 | 7 | 8 

AI is thinking...
AI played position 4

Current Board:
-------------
 0 | 1 | O 
-----------
 3 | X | 5 
-----------
 6 | 7 | 8 

Your turn (O):
That position is already taken! Choose another.
That position is already taken! Choose another.
You played position 1

Current Board:
-------------
 0 | O | O 
-----------
 3 | X | 5 
-----------
 6 | 7 | 8 

AI is thinking...
AI played position 0

Current Board:
-------------
 X | O | O 
-----------
 3 | X | 5 
-----------
 6 | 7 | 8 

Your turn (O):
That position is already taken! Choose another.
You played position 3

Cu